# Week 5 — Integration and Final Backtest  

Up to now, we have deliberately kept things **static**.  
Given predicted returns and a covariance matrix, we computed an optimal portfolio and
analyzed its properties.

In this week, we finally answer the natural follow-up question:

> What actually happens if we *run* this optimizer over time?

---

### References
- Portfolio Optimization Book - Ch 6 - Sections 1, 2 for defining the formulae for wealth evolution.
- ["Markowitz Portfolio Construction at Seventy" by Boyd et al.](https://web.stanford.edu/~boyd/papers/pdf/markowitz.pdf) - For defining the transaction costs and detailed treatment of other costs that we skipped over.

## 1. Wealth dynamics and portfolio returns (theory)

We now make precise what it means to *run* a portfolio over time.

At each time $t$, asset prices change and the value of the portfolio evolves accordingly.
There are two equivalent but conceptually distinct ways to think about this:
in terms of **capital allocations** and in terms of **normalized weights**.
In practice, optimization works with normalized weights, while wealth evolution happens in capital.

---

### Portfolio return

Let:
- $r_t^{\text{lin}} \in \mathbb{R}^n$ be the vector of **linear returns** from time $t$ to $t+1$,
- $w_{t-1} \in \mathbb{R}^n$ be the **normalized portfolio weights** held during period $t$.

The portfolio return over period $t$ is

$$
R_t^{\text{portf}} = w_{t-1}^\top r_t^{\text{lin}}.
$$

> This equation explains why portfolio returns are **linear** in asset returns.
> It also fixes the time index: weights chosen using information up to $t-1$ are applied to returns at $t$.
> Using $w_t^\top r_t$ would introduce look-ahead bias.

---



### Wealth (NAV) evolution

Let $V_t$ denote the **net asset value (NAV)** or wealth at time $t$.

Ignoring transaction costs for the moment, wealth evolves as

$$
V_t = V_{t-1} \bigl(1 + R_t^{\text{portf}}\bigr).
$$

Iterating this recursion yields

$$
V_t
=
V_0 \prod_{s=1}^t \bigl(1 + w_{s-1}^\top r_s^{\text{lin}}\bigr).
$$

> This multiplicative structure is why backtests must be run over time.
> Portfolio decisions interact *across* periods through compounding.

---



### Expected return and variance (single period)

Conditioned on information available at time $t-1$, the first two moments of the portfolio return are

$$
\mathbb{E}[R_t^{\text{portf}}] = w_{t-1}^\top \mu,
\qquad
\mathrm{Var}(R_t^{\text{portf}}) = w_{t-1}^\top \Sigma w_{t-1}.
$$

These are exactly the quantities used in mean–variance optimization.
In Week 5 we repeatedly recompute these quantities over time using rolling estimates.

---


## 2. What is a strategy? (and what is being reinvested)

So far we have talked about portfolios, weights, and wealth, but not explicitly defined
what we mean by a **strategy**.

Informally:

> A **strategy** is a rule that specifies how portfolio weights are chosen over time
> and how gains or losses are handled as wealth evolves.

This definition has two distinct components:
1. **Decision rule:** how we choose weights $w_t$ using past data.
2. **Reinvestment rule:** how portfolio value is treated after gains or losses.

The first part is handled by prediction and optimization.
The second part is about *how capital evolves*.

---

### Full reinvesting vs constant investment

The book distinguishes between two common conventions.

#### Full reinvesting (self-financing strategy)

In a **fully reinvested strategy**, all gains and losses are reinvested back into the portfolio.

This is exactly the wealth dynamics we have used so far.

If $V_t$ is the wealth at time $t$, then
$$
V_{t+1} = V_t \bigl(1 + w_t^\top r_{t+1}^{\text{lin}}\bigr).
$$

Here:
- portfolio weights $w_t$ are fractions of *current* wealth $V_t$,
- profits increase future position sizes,
- losses reduce future position sizes.

> This is the standard assumption in portfolio theory and backtesting.
> It corresponds to a **self-financing strategy**.

Most academic and practical backtests use full reinvesting.

#### Constant investment (fixed notional exposure)

In a **constant investment strategy**, the same fixed amount of capital is deployed
each period, regardless of past gains or losses.

Conceptually:
- wealth is tracked for accounting,
- but the portfolio each period is constructed using a fixed reference capital.

Returns are aggregated additively rather than multiplicatively.

> This convention is sometimes useful for isolating per-period performance,
> but it does **not** model compounding and is less realistic for long-horizon analysis.

---

### Which convention are we using?

In this notebook, we use **full reinvesting** throughout because:
- it matches how real portfolios evolve,
- it captures compounding effects,
- it aligns naturally with normalized portfolio weights.

When we say “run a strategy over time”, we mean:
> repeatedly choose weights and fully reinvest the resulting gains or losses.

This clarifies the interpretation of the wealth process $V_t$ used in the backtest loop.

---


## 3. Rebalancing and transaction costs (minimal model)

Because asset prices change, portfolio weights **drift** even if no trades are made.
To maintain a target portfolio, we must rebalance, which incurs costs.

There are various models for the transaction cost, depending the kind of instrument being traded, the broker etc. We use a simple model here: transaction cost is a linear function of the sum of absolute deviations on portfolio weights.

---

### Turnover-based transaction costs

Let:
- $w_{t-1}$ be the portfolio before rebalancing,
- $w_t$ be the new target portfolio.

Define turnover as

$$
\text{turnover}_t = \sum_i |w_{t,i} - w_{t-1,i}|.
$$

With proportional transaction cost rate $c > 0$, the cost paid is (this is why we need the cash term as separate from the weights vector; that is used to fulfill these costs)

$$
\text{cost}_t = c \cdot \text{turnover}_t.
$$

The wealth update becomes

$$
V_t
=
V_{t-1} \bigl(1 + w_{t-1}^\top r_t^{\text{lin}}\bigr)
-
V_{t-1} \cdot \text{cost}_t.
$$

> Transaction costs reduce returns but do not affect the variance term.
> This asymmetry is why high-turnover strategies often look good ex ante but perform poorly when implemented.

---



## 4. Biases and time indexing

Two points must be respected for a backtest to be valid:

> **Temporal causality:**  
> Weights applied to returns at time $t$ must be computed using information available up to $t-1$.

> **Survivorship bias:**  
> The asset universe must reflect what was tradable at time $t$, not what survives to the end of the dataset.

Violating either produces optimistic but invalid results.

---



## 5. Pseudo-algorithm: full backtest loop

Below is the conceptual algorithm implemented in Week 5.

The basic idea is:

> make a prediction -> optimize the portfolio -> track your wealth -> periodically (monthly/weekly) rebalance the portfolio -> repeat

---

### Inputs

- Asset return series $\{r_t^{\text{lin}}\}_{t=1}^T$
- Estimation window length $L$
- Rebalancing period $K$ (e.g. $K=5$ for weekly, $K=21$ for monthly)
- Initial wealth $V_0$
- Optimization routine $\texttt{Optimize}(\hat\mu, \hat\Sigma)$
- Transaction cost rate $c$

---

### Initialization

- Set $V_0 = 1$
- Choose initial portfolio weights $w_0$
- Set last rebalanced weights $\tilde w_0 = w_0$

---

### Main loop

For $t = L, \dots, T-1$:

1. **Check if rebalancing date**

   If $t \bmod K = 0$:

   a. **Estimate parameters**
   $$
   (\hat\mu_t, \hat\Sigma_t)
   \leftarrow
   \text{estimate from } \{r_{t-L}, \dots, r_{t-1}\}. \quad\quad \text{(Weeks 2-3)}
   $$

   b. **Compute target portfolio**
   $$
   w_t \leftarrow \texttt{Optimize}(\hat\mu_t, \hat\Sigma_t). \quad\quad \text{(Week 4)}
   $$

   c. **Compute turnover and transaction cost**
   $$
   \text{turnover}_t = \sum_i |w_{t,i} - \tilde w_{t-1,i}|,
   \qquad
   \text{cost}_t = c \cdot \text{turnover}_t.
   $$

   d. **Update last rebalanced weights**
   $$
   \tilde w_t \leftarrow w_t.
   $$

   Otherwise (no rebalancing):

   $$
   w_t \leftarrow \tilde w_{t-1},
   \qquad
   \text{cost}_t = 0.
   $$

---

2. **Apply returns and update wealth**

   The portfolio held during period $t$ earns return
   $$
   R_t^{\text{portf}} = w_t^\top r_{t+1}^{\text{lin}}.
   $$

   Wealth evolves as
   $$
   V_{t+1}
   =
   V_t (1 + R_t^{\text{portf}})
   -
   V_t \cdot \text{cost}_t.
   $$

---

3. **Record diagnostics**

- Wealth $V_{t+1}$
- Portfolio return $R_t^{\text{portf}}$
- Weights $w_t$
- Turnover $\text{turnover}_t$

---

### Outputs

- Wealth trajectory $\{V_t\}_{t=0}^T$
- Portfolio return series
- Weight paths
- Turnover and transaction cost time series

---

### Remarks

> Most realistic strategies rebalance weekly or monthly, not daily.